# Part 1

## Import libraries

In [57]:
import requests
import lxml.html as lh
import pandas as pd

## Scrape table from wikipeida

In [58]:
my_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(my_url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content().rstrip('\n')
    col.append((name,[]))


In [59]:
for j in range(1,len(tr_elements)):

    T=tr_elements[j]
    

    if len(T)!=3:
        break
    

    i=0
    

    for t in T.iterchildren():
        data=t.text_content().rstrip('\n') 

        if i>0:

            try:
                data=int(data)
            except:
                pass

        col[i][1].append(data)

        i+=1

In [60]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [61]:
# there is one row at the bottom of this dataframe from another table. Remove it
df=df.drop([180])
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Remove NA from Borough

In [62]:
df=df[df.Borough!='Not assigned'].reset_index(drop=True)

In [63]:
df[df.Neighbourhood=='Not assigned']
# There is no 'Not assigned' neighbourhood

,Postal Code,Borough,Neighbourhood


In [64]:
df.shape

(103, 3)

# Part 2

In [65]:
"""
!conda install -c conda-forge geocoder
import geocoder # import geocoder
import numpy as np
df[['latitude','longitude']]=pd.DataFrame([[np.nan,np.nan]])

for i in range(0,len(df['Postal Code'])-1):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    df['latitude'][i] = lat_lng_coords[0]
    df['longitude'][i] = lat_lng_coords[1]

I tried to install geocoder but failed
"""   


"\n!conda install -c conda-forge geocoder\nimport geocoder # import geocoder\nimport numpy as np\ndf[['latitude','longitude']]=pd.DataFrame([[np.nan,np.nan]])\n\nfor i in range(0,len(df['Postal Code'])-1):\n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n\n    df['latitude'][i] = lat_lng_coords[0]\n    df['longitude'][i] = lat_lng_coords[1]\n\nI tried to install geocoder but failed\n"

In [66]:
# get latitude and longitude data of postal code from this link https://cocl.us/Geospatial_data
!wget -q -O 'geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [67]:
geo_data = pd.read_csv('geospatial_data.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [68]:
df = pd.merge(df,geo_data,how='left',on='Postal Code')

In [69]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part 3 Segmenting and Clustering Neighbourhood

## Import libraries

In [74]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# failed to install folium...

## Subset with Boroughs containing Toronto

In [80]:
df_new=df[df['Borough'].str.contains("Toronto")]
df_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


#### Use geopy library to get the latitude and longitude values of Toronto.


In [82]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [83]:
"""
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto
"""
# not able to create the map due to failure of installing folium package

NameError: name 'folium' is not defined

#### Define Foursquare Credentials and Version


In [92]:
CLIENT_ID = 'LUZAON1NYWFA0QTJAOQBU4BGXZYZ0ZX1VVCG3RXBQOVUI1A0' # your Foursquare ID
CLIENT_SECRET = 'Q4QRCLGQ2LGI0SK0FV5L5R3Y0Y2LIQPKQPGVLDPVAMG3ZSBR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LUZAON1NYWFA0QTJAOQBU4BGXZYZ0ZX1VVCG3RXBQOVUI1A0
CLIENT_SECRET:Q4QRCLGQ2LGI0SK0FV5L5R3Y0Y2LIQPKQPGVLDPVAMG3ZSBR


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
M_names = df_new['Neighborhood']
M_lat = df_new['Latitude']
M_lon = df_new['Longitude']
toronto_venues = getNearbyVenues(names=M_names,latitudes=M_lat,longitudes=M_lon)

KeyError: 'Neighborhood'